In [1]:
# path variables
import sys
import json
project_path = '/Users/naresh/Downloads/ds_models/onboarding_fraud_model_v2/'
sys.path.insert(0, project_path+'config')

# core libraries
import datetime
import pandas as pd
import numpy as np
import warnings

from config import SQLQuery
from collections import Counter
from tqdm import tqdm
from collections import defaultdict

warnings.filterwarnings("ignore")

In [2]:
# q = SQLQuery('snowflake')

#### Analyzing the sardine data

In [3]:
# df_sardine = q("""

# with owners_data_tmp as (
# select b.application_id, a.*, rank() over(partition by application_id order by a.created_at) as rk
# from "FIVETRAN_DB"."PROD_NOVO_ONBOARDING_PUBLIC"."OWNER_RISK_DETAILS" a
# inner join FIVETRAN_DB.PROD_NOVO_ONBOARDING_PUBLIC.OWNERS b on a.owners_id=b.id
# )

# select b.*
# from prod_db.data.businesses a
# inner join (select * from owners_data_tmp where rk=1) b on a.application_id=b.application_id
# where 1=1 
# and a.account_create_date between '2023-01-01' and '2023-12-31'
# and a.business_type ilike '%sole%'

# """)

In [4]:
df_sardine = pd.read_pickle(project_path+'data/df_sardine.pkl')

In [5]:
df_sardine.shape

(23333, 12)

In [6]:
json.loads(df_sardine['meta'][0].lower()).keys()

dict_keys(['checkpointdata', 'checkpoints', 'customer', 'device', 'level', 'rules', 'sessionkey', 'status'])

In [7]:
json.loads(df_sardine['meta'][6].lower())['customer']

{'address': {'validity': 'unknown'},
 'level': 'low',
 'score': 4,
 'signals': [{'key': 'addressrisklevel', 'value': 'low'},
  {'key': 'emaildomainlevel', 'value': 'low'},
  {'key': 'emaillevel', 'value': 'low'},
  {'key': 'phonelevel', 'value': 'low'},
  {'key': 'manualreview', 'value': 'false'}]}

In [8]:
json.loads(df_sardine['meta'][0].lower())['device']

{'attributes': {'browser': ['mobile safari'],
  'model': ['iphone'],
  'os': ['ios']},
 'behaviorbiometrics': {'hesitationpercentile': {'ltm': 0, 'nonltm': 0},
  'numcontextswitchevents': 0,
  'numdistractionevents': 0,
  'pagefieldltm': {'absolute': {'hesitationpercentage': 0,
    'isltm': False,
    'numautofillevents': 0,
    'numclipboardevents': 0,
    'numcopypasteevents': 0,
    'numexpertkeyevents': 0,
    'timespendinms': 0,
    'timespendinmsevents': None},
   'average': {'hesitationpercentage': 0,
    'isltm': False,
    'numautofillevents': 0,
    'numclipboardevents': 0,
    'numcopypasteevents': 0,
    'numexpertkeyevents': 0,
    'timespendinms': 0,
    'timespendinmsevents': None},
   'percent': {'hesitationpercentage': 0,
    'isltm': False,
    'numautofillevents': 0,
    'numclipboardevents': 0,
    'numcopypasteevents': 0,
    'numexpertkeyevents': 0,
    'timespendinms': 0,
    'timespendinmsevents': None}},
  'pagefieldnonltm': {'absolute': {'hesitationpercentage'

In [9]:
json.loads(df_sardine['meta'][0].lower())['device'].keys()

dict_keys(['attributes', 'behaviorbiometrics', 'checkpoints', 'confidencescore', 'devicereputation', 'fingerprint', 'fingerprintconfidencescore', 'id', 'ipaddresses', 'iplocation', 'level', 'sessionkey', 'signals'])

In [10]:
json.loads(df_sardine['meta'][1].lower())['device']['behaviorbiometrics'].keys()

dict_keys(['fields', 'flowtimesinms', 'hesitationpercentile', 'numcontextswitchevents', 'numdistractionevents', 'pagefieldltm', 'pagefieldnonltm', 'totaltimespentinms', 'typingspeed'])

In [11]:
json.loads(df_sardine['meta'][1].lower())['device']['attributes']

{'browser': ['chrome'], 'model': [''], 'os': ['windows']}

In [12]:
json.loads(df_sardine['meta'][1].lower())['device']['confidencescore']

0.84242

In [13]:
json.loads(df_sardine['meta'][10].lower())['device']['devicereputation']

'medium_risk'

In [14]:
json.loads(df_sardine['meta'][10].lower())['device']['signals']

[{'key': 'trueos', 'value': 'mac/ios'},
 {'key': 'deviceagehours', 'value': '0'},
 {'key': 'trueip', 'value': '166.199.3.145'},
 {'key': 'vpn', 'value': 'low'},
 {'key': 'proxy', 'value': 'low'},
 {'key': 'remotesoftware', 'value': 'false'},
 {'key': 'remotesoftwarelevel', 'value': 'low'},
 {'key': 'osanomaly', 'value': 'low'},
 {'key': 'emulator', 'value': 'false'},
 {'key': 'tamperedapp', 'value': ''},
 {'key': 'iptype', 'value': 'mobile isp'},
 {'key': 'sessionipcount', 'value': '2'},
 {'key': 'sessionipcountrycount', 'value': '1'},
 {'key': 'remotesessionlevel', 'value': ''},
 {'key': 'accountdeviceid', 'value': '11331843-f5ea-411b-969a-5981e427b85f'},
 {'key': 'behaviorbiometriclevel', 'value': 'low'}]

#### Data Extraction Starts

In [15]:
keys_mapping = {
    'otp':'otp', 'phone otp':'phone_otp', 'business_type':'business_type',
    'day':'day', 'month':'month', 'year':'year', 
    'phone':'phone', '(201) 555-5555':'phone',
    'ssn':'ssn', 'enter your ssn':'ssn', 'ein':'ein', 'ein_disclosure':'ein_disclosure', 
    'last_name':'last_name', 'enter your last name':'last_name',
    'first_name':'first_name', 'enter your first name':'first_name',
    'middle_name':'middle_name', 'enter your middle name':'middle_name',
    
    'purpose_of_account':'purpose_of_account', 'purpose_option':'purpose_of_account',
    
    'outgoing_wire_transfers':'outgoing_wire_transfers', 'business_questions#outgoing_wire_transfers':'outgoing_wire_transfers',
    'estimated_monthly_revenue':'estimated_monthly_revenue', 'business_questions#estimated_monthly_revenue':'estimated_monthly_revenue',
    'incoming_ach_payments':'incoming_ach_payments', 'business_questions#incoming_ach_payments':'incoming_ach_payments',
    'check_deposit_amount':'check_deposit_amount', 'business_questions#check_deposit_amount':'check_deposit_amount',
    'outgoing_ach_and_checks':'outgoing_ach_and_checks', 'business_questions#outgoing_ach_and_checks':'outgoing_ach_and_checks',
    'incoming_wire_transfer':'incoming_wire_transfer', 'business_questions#incoming_wire_transfer':'incoming_wire_transfer',
    
    'number_of_employees':'number_of_employees', 'enter estimated number of employees':'number_of_employees',
    'website':'website', 'yourbusiness.com (optional)':'website',
    # 'referral_code':'referral_code', 
    'company_name':'company_name', 'business#company_name':'company_name',
    'business_pitch':'business_pitch',
    'eg. we provide a lunch catering service for companies around 100 employees':'business_pitch',
    "par exemple.  nous proposons un service de restauration pour le déjeuner aux entreprises d'environ 100 salariés":'business_pitch',
    "eg. we provide a lunch catering service for companies around 100 employees, so they don’t need to organize their own kitchen services in house. we operate exclusively in new york city but have plans to eventually setup operations in boston as well. our main source of revenue comes from selling monthly subscriptions for our luch catering service, with choice of an entry level, or premium package.":'business_pitch' 
}

In [16]:
# Function to convert list of dicts to dicts with 'name' key values
def name_list_of_dicts_by_key(list_of_dicts, key_name):
    # Initialize an empty dictionary to hold the named result
    named_dict = {}
    
    # Iterate over each dictionary in the list
    for d in list_of_dicts:
        # Extract the value for the specified key to use as the new key
        if key_name in d:
            new_key = d[key_name]
            named_dict[new_key] = d
        else:
            print(f"Warning: Key '{key_name}' not found in dictionary {d}")
    
    return named_dict

# Function to replace 'name' key values
def replace_name_values(data, keys_mapping_dict):
    for item in data:
        try:
            item['name'] = keys_mapping_dict[item['name']]
        except:
            continue
    return data
    
# Function to handle duplicate keys
def append_duplicate_keys(dicts, key_field):
    result = defaultdict(list)
    for d in dicts:
        key = d[key_field]
        result[key].append(d)
    return result

# Function to append values of the same keys into a list
def append_values_of_same_keys(dict_of_lists, key_name):
    combined_dict = defaultdict(list)
    for item in dict_of_lists[key_name]:
        for key, value in item.items():
            combined_dict[key].append(value)
    return dict(combined_dict)


In [17]:
# Initialize Columns to Extract
device_biometr_all_variants_keys_list = list(keys_mapping.keys())

device_biometr_keys = ['business_type', 'first_name', 'middle_name', 'last_name', 'incoming_wire_transfer', 'check_deposit_amount',
                        'incoming_ach_payments', 'estimated_monthly_revenue', 'outgoing_wire_transfers', 'outgoing_ach_and_checks',
                        'purpose_of_account', 'otp', 'phone_otp', 'ssn', 'business_pitch', 'number_of_employees','website',
                        'day','month','year','referral_code','company_name']
device_biometr_fields = ['numautofillevents','numclipboardevents','numcopypasteevents','numexpertkeyevents','timespendinms',
                         'timespendinmsevents']
device_typing_fields = ['downspeed','firstkeydownspeed','firstkeyspeed','firstkeyupspeed','mediandowntime','medianuptime',
                        'secondkeydownspeed','secondkeyspeed','secondkeyupspeed','typingaccuracy','typingspeed','upspeed']
device_customer_fields = ['address_validity','level','score','addressrisklevel','emaildomainlevel','emaillevel','phonelevel']
device_other_fields = ['trueos','deviceagehours','trueip','vpn','proxy','remotesoftware','remotesoftwarelevel','osanomaly','emulator',
                       'tamperedapp','iptype','sessionipcount','sessionipcountrycount','remotesessionlevel','accountdeviceid',
                       'behaviorbiometriclevel']
device_other_attribute_columns = ['sdn_attribute_browser','sdn_attribute_model','sdn_attribute_os','sdn_device_confidencescore',
                                  'sdn_device_devicereputation','sdn_device_fingerprintconfidencescore']

device_biometr_columns = [f"sdn_biometr_{key}_{field}" for key in device_biometr_keys for field in device_biometr_fields]
device_typing_columns = [f"sdn_typing_{field}" for field in device_typing_fields]
device_customer_columns = [f"sdn_customer_{field}" for field in device_customer_fields]
device_other_columns = [f"sdn_other_{field}" for field in device_other_fields]
device_other_attribute_columns = device_other_attribute_columns

# Initialize DataFrames
device_biometr_df = pd.DataFrame(columns=device_biometr_columns)
device_typing_df = pd.DataFrame(columns=device_typing_columns)
device_customer_df = pd.DataFrame(columns=device_customer_columns)
device_other_df = pd.DataFrame(columns=device_customer_columns)
device_other_attribute_df = pd.DataFrame(columns=device_other_attribute_columns)

device_biometr_except_list = []
device_typing_except_list = []
device_customer_except_list = []
device_other_except_list = []
device_other_attribute_except_list = []


In [18]:

def extract_device_biometr_signals(row):
    try:
        tmp_dict = {}
        named_dict = {}
        # Extracting Device Relevant data
        device_data = json.loads(row['meta'].lower()).get('device', {}).get('behaviorbiometrics', {}).get('fields', [])
        device_data = replace_name_values(device_data, keys_mapping)
        named_dict_dup_keys = append_duplicate_keys(device_data, 'name')
        for key in named_dict_dup_keys.keys():
            if key in device_biometr_all_variants_keys_list:
                key1 = keys_mapping[key]
                named_dict[key1] = append_values_of_same_keys(named_dict_dup_keys, key1)
                tmp_dict[f"sdn_biometr_{key1}_numautofillevents"] = str(named_dict.get(key1, {}).get('numautofillevents', None))
                tmp_dict[f"sdn_biometr_{key1}_numclipboardevents"] = str(named_dict.get(key1, {}).get('numclipboardevents', None))
                tmp_dict[f"sdn_biometr_{key1}_numcopypasteevents"] = str(named_dict.get(key1, {}).get('numcopypasteevents', None))
                tmp_dict[f"sdn_biometr_{key1}_numexpertkeyevents"] = str(named_dict.get(key1, {}).get('numexpertkeyevents', None))
                tmp_dict[f"sdn_biometr_{key1}_timespendinms"] = str(named_dict.get(key1, {}).get('timespendinms', None))
                tmp_dict[f"sdn_biometr_{key1}_timespendinmsevents"] = str(named_dict.get(key1, {}).get('timespendinmsevents', None))

        return tmp_dict
    except:
        device_except_list.append(row.name)
        return pd.Series([pd.np.nan]*len(device_biometr_columns), index=device_biometr_columns)

def extract_device_typing_signals(row):
    try:
        tmp_dict = {}
        # Extracting Typing Speed data
        typing_data = json.loads(row['meta'].lower()).get('device',{}).get('behaviorbiometrics',{}).get('typingspeed', [])
        tmp_dict['sdn_typing_downspeed'] = typing_data.get('downspeed', None)
        tmp_dict['sdn_typing_firstkeydownspeed'] = typing_data.get('firstkeydownspeed', None)
        tmp_dict['sdn_typing_firstkeyspeed'] = typing_data.get('firstkeyspeed', None)
        tmp_dict['sdn_typing_firstkeyupspeed'] = typing_data.get('firstkeyupspeed', None)
        tmp_dict['sdn_typing_mediandowntime'] = typing_data.get('mediandowntime', None)
        tmp_dict['sdn_typing_medianuptime'] = typing_data.get('medianuptime', None)
        tmp_dict['sdn_typing_secondkeydownspeed'] = typing_data.get('secondkeydownspeed', None)
        tmp_dict['sdn_typing__secondkeyspeed'] = typing_data.get('secondkeyspeed', None)
        tmp_dict['sdn_typing_secondkeyupspeed'] = typing_data.get('secondkeyupspeed', None)
        tmp_dict['sdn_typing_typingaccuracy'] = typing_data.get('typingaccuracy', None)
        tmp_dict['sdn_typing_typingspeed'] = typing_data.get('typingspeed', None)
        tmp_dict['sdn_typing_upspeed'] = typing_data.get('upspeed', None)
        return tmp_dict
    except:
        device_typing_except_list.append(row.name)
        return pd.Series([pd.np.nan]*len(device_typing_columns), index=device_typing_columns)


def extract_device_customer_signals(row):
    try:
        signals_dict = {}
        customer_data = json.loads(row['meta'].lower()).get('customer', {})
        signals_dict['sdn_customer_address_validity'] = customer_data.get('address', {}).get('validity', None)
        signals_dict['sdn_customer_level'] = customer_data.get('level', None)
        signals_dict['sdn_customer_score'] = customer_data.get('score', None)

        named_dict2 = name_list_of_dicts_by_key(customer_data.get('signals', []), 'key')
        signals_dict['sdn_customer_addressrisklevel'] = named_dict2.get('addressrisklevel', {}).get('value', None)
        signals_dict['sdn_customer_emaildomainlevel'] = named_dict2.get('emaildomainlevel', {}).get('value', None)
        signals_dict['sdn_customer_emaillevel'] = named_dict2.get('emaillevel', {}).get('value', None)
        signals_dict['sdn_customer_phonelevel'] = named_dict2.get('phonelevel', {}).get('value', None)
        return signals_dict
    except:
        customer_except_list.append(row.name)
        return pd.Series([pd.np.nan]*len(device_customer_columns), index=device_customer_columns)


def extract_device_other_signals(row):
    try:
        tmp_dict = {}
        # Extracting data
        device_data = json.loads(row['meta'].lower()).get('device', {}).get('signals', [])
        named_dict = name_list_of_dicts_by_key(device_data, 'key')

        tmp_dict['sdn_other_trueos'] = named_dict.get('trueos', {}).get('value', None)
        tmp_dict['sdn_other_deviceagehours'] = named_dict.get('deviceagehours', {}).get('value', None)
        tmp_dict['sdn_other_trueip'] = named_dict.get('trueip', {}).get('value', None)
        tmp_dict['sdn_other_vpn'] = named_dict.get('vpn', {}).get('value', None)
        tmp_dict['sdn_other_proxy'] = named_dict.get('proxy', {}).get('value', None)
        tmp_dict['sdn_other_remotesoftware'] = named_dict.get('remotesoftware', {}).get('value', None)
        tmp_dict['sdn_other_remotesoftwarelevel'] = named_dict.get('remotesoftwarelevel', {}).get('value', None)
        tmp_dict['sdn_other_osanomaly'] = named_dict.get('osanomaly', {}).get('value', None)
        tmp_dict['sdn_other_emulator'] = named_dict.get('emulator', {}).get('value', None)
        tmp_dict['sdn_other_tamperedapp'] = named_dict.get('tamperedapp', {}).get('value', None)
        tmp_dict['sdn_other_iptype'] = named_dict.get('iptype', {}).get('value', None)
        tmp_dict['sdn_other_sessionipcount'] = named_dict.get('sessionipcount', {}).get('value', None)
        tmp_dict['sdn_other_sessionipcountrycount'] = named_dict.get('sessionipcountrycount', {}).get('value', None)
        tmp_dict['sdn_other_remotesessionlevel'] = named_dict.get('remotesessionlevel', {}).get('value', None)
        tmp_dict['sdn_other_accountdeviceid'] = named_dict.get('accountdeviceid', {}).get('value', None)
        tmp_dict['sdn_other_behaviorbiometriclevel'] = named_dict.get('behaviorbiometriclevel', {}).get('value', None)
        return tmp_dict
    except:
        device_other_except_list.append(row.name)
        return pd.Series([pd.np.nan]*len(device_other_columns), index=device_other_columns)


def extract_device_other_attribute_signals(row):
    try:
        tmp_dict = {}
        device_data_dict = json.loads(row['meta'].lower()).get('device', {}).get('attributes', [])
        
        tmp_dict['sdn_attribute_browser'] = device_data_dict.get('browser', None)
        tmp_dict['sdn_attribute_model'] = device_data_dict.get('model', None)
        tmp_dict['sdn_attribute_os'] = device_data_dict.get('os', None)
        tmp_dict['sdn_device_confidencescore'] = json.loads(row['meta'].lower()).get('device', {}).get('confidencescore', None)
        tmp_dict['sdn_device_devicereputation'] = json.loads(row['meta'].lower()).get('device', {}).get('devicereputation', None)
        tmp_dict['sdn_device_fingerprintconfidencescore'] = json.loads(row['meta'].lower()).get('device', {}
                                                                                               ).get('fingerprintconfidencescore', None)
        
        return tmp_dict
    except:
        device_other_attribute_except_list.append(row.name)
        return pd.Series([pd.np.nan]*len(device_other_attribute_columns), index=device_other_attribute_columns)



In [19]:
# df_sar_tmp = df_sardine[:1000]

In [20]:

# Apply the functions to the DataFrame
device_biometr_df = df_sardine.apply(extract_device_biometr_signals, axis=1, result_type='expand')
device_typing_df = df_sardine.apply(extract_device_typing_signals, axis=1, result_type='expand')
device_customer_df = df_sardine.apply(extract_device_customer_signals, axis=1, result_type='expand')
device_other_df = df_sardine.apply(extract_device_other_signals, axis=1, result_type='expand')
device_other_attribute_df = df_sardine.apply(extract_device_other_attribute_signals, axis=1, result_type='expand')

combined_df = pd.concat([df_sardine,device_biometr_df,device_typing_df,device_customer_df,device_other_df,device_other_attribute_df], axis=1)

In [21]:
device_biometr_df.shape, device_typing_df.shape, device_customer_df.shape, device_other_df.shape

((23333, 138), (23333, 13), (23333, 7), (23333, 16))

In [22]:
# name_list_of_dicts_by_key(json.loads(df_sardine['meta'][1].lower())['device']['behaviorbiometrics']['fields'], 'name')

In [23]:
# device_biometr_df[device_biometr_df.sdn_biometr_business_pitch_numexpertkeyevents.isnull()]
# # device_other_df

In [24]:
# device_biometr_df[device_biometr_df.columns[device_biometr_df.columns.str.startswith('sdn_biometr_business_pitch')==True].to_list()]

In [25]:
# device_data = json.loads(df_sardine['meta'][10].lower()).get('device', {}).get('signals', [])
# named_dict = name_list_of_dicts_by_key(device_data, 'key')


In [27]:
combined_df.to_pickle(project_path+'data/sardine_data_extracted.pkl')